In [19]:
import sys
import numpy as np
sys.path.append("../utils")
from mujoco_parser import MuJoCoParserClass
from util import MultiSliderClass,get_idxs
print ("Ready.")

Ready.


In [20]:
xml_path = '../asset/smplh_rig/scene_smplh_rig.xml'
env = MuJoCoParserClass(name='SMPL-H Rig',rel_xml_path=xml_path,VERBOSE=False)
for body_name in env.body_names:
    if body_name in ['world']: continue
    for geom_idx in env.get_geom_idxs_from_body_name(body_name=body_name):
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,1]
print ("Done.")

Done.


In [21]:
q = np.zeros(env.model.nq)
q[2] = 0.956
q[26] = -np.pi/2
env.forward(q)

wrist_pos = env.get_p_body('left_wrist')
wrist_rot = env.get_R_body('left_wrist')
JOINT_NAMES = ['l_shoulder1','l_shoulder2','l_shoulder3','l_elbow','l_wrist1','l_wrist2','l_wrist3']
idxs = get_idxs(env.rev_joint_names,JOINT_NAMES)
q_mins_arms,q_maxs_arms = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
joint_idxs_fwd_arms = env.get_idxs_fwd(joint_names=JOINT_NAMES)
joint_idxs_jac_arms = env.get_idxs_jac(joint_names=JOINT_NAMES)

In [22]:
sliders = MultiSliderClass(
    n_slider      = 3,
    title         = 'Sliders for SMPL-H Wrist Control',
    window_width  = 500,
    window_height = 800,
    x_offset      = 50,
    y_offset      = 100,
    slider_width  = 350,
    label_texts   = ['x','y','z'],
    slider_mins   = wrist_pos + [-1,-1,-1],
    slider_maxs   = wrist_pos + [1,1,1],
    slider_vals   = wrist_pos,
    resolution    = 0.01,
    VERBOSE       = False,
)

env.init_viewer(
    viewer_title='SMPL-H Rig',viewer_width=1200,viewer_height=800,
    viewer_hide_menus=True,
)
env.update_viewer(
    azimuth=180,distance=1.24,elevation=-32,lookat=[0.25,0.5,1.73],
    VIS_TRANSPARENT=True,
)

env.reset()

while env.is_viewer_alive():
    sliders.update()
    tp = sliders.get_slider_values()

    q_arms = env.data.qpos[joint_idxs_fwd_arms]
    ik_p_target = tp
    ik_R_target = wrist_rot
    J,ik_err = env.get_ik_ingredients(body_name='left_wrist',p_trgt=ik_p_target,R_trgt=ik_R_target)
    dq = env.damped_ls(J,ik_err,stepsize=1,eps=1e-2,th=np.radians(1.0))
    q_arms += dq[joint_idxs_jac_arms]
    q_arms = np.clip(q_arms,q_mins_arms,q_maxs_arms)
    env.forward(q=q_arms,joint_idxs=joint_idxs_fwd_arms)
    wp = env.get_p_joint('l_wrist1')

    env.plot_T(p=tp,R=wrist_rot,PLOT_AXIS=True,axis_len=0.1,axis_width=0.005,
               label='Wrist Target: [%.2f %.2f %.2f]\n Wrist Position: [%.2f %.2f %.2f]\n Distance: %.4f' %
               (tp[0],tp[1],tp[2],wp[0],wp[1],wp[2],np.linalg.norm(tp-wp)))
    env.plot_T(p=env.get_p_body('left_wrist'),R=env.get_R_body('left_wrist'),PLOT_AXIS=True,axis_len=0.1,axis_width=0.005)
    env.render()

env.close_viewer()
sliders.close()

Pressed ESC
Quitting.
